# Predict Radiative heating corresponding to given parameters

## Import package

In [33]:
import h5py
import numpy as np
from matplotlib import pyplot as plt

from metpy.calc import pressure_to_height_std
from metpy.units import units
from scipy.interpolate import interp1d

## Load data

In [34]:
# Load LRF
## Load LRF for T, q
with h5py.File("/work/b11209013/2025_Research/MSI/Rad_Stuff/LRF.h5", "r") as f:
    q_lw_lrf = np.array(f.get("q_lw"))[::-1, ::-1]
    q_sw_lrf = np.array(f.get("q_sw"))[::-1, ::-1] # 1000 -> 100
    t_lw_lrf = np.array(f.get("t_lw"))[::-1, ::-1]
    t_sw_lrf = np.array(f.get("t_sw"))[::-1, ::-1]

## Load LRF for w
with h5py.File("/home/b11209013/2025_Research/Obs/Files/ERA5/LRF_w.h5", "r") as f:
    w_lw_lrf = np.array(f.get("LW_LRF"))
    w_sw_lrf = np.array(f.get("SW_LRF"))
    eof = np.array(f.get("EOF"))


# load regressed moisture profile
q_reg = np.loadtxt("/work/b11209013/2025_Research/MSI/Rad_Stuff/mean_corr_moisture.txt")[::-1]
# 1000 -> 100

# Load vertical modes
with h5py.File("/home/b11209013/2025_Research/Kuang2008/data/vertical_mode.h5", "r") as f:
    G1 = np.array(f.get("G1")).squeeze() #1000 -> 100
    G2 = np.array(f.get("G2")).squeeze()
    z  = np.array(f.get("z"))

# Load background density profile
with h5py.File("/home/b11209013/2025_Research/Kuang2008/data/background.h5","r") as f:
    temp_profile = np.array(f.get("T0")).squeeze(); #
    rho_profile = np.array(f.get("ρ0")).squeeze(); # 1000 -> 100

## Interpolation

In [35]:
levels = np.linspace(1000, 100, 37)
z_std = np.array(pressure_to_height_std((levels).astype(int) * units.hPa)) *1000.0

# interpolate moisture profile
mean_moist_itp = np.interp(
    levels[::-1], np.array([250, 500, 700, 850, 925, 1000]), q_reg[::-1]
)[::-1]


# Interpolate vertical mode
G1_itp = interp1d(z, G1, kind="linear", bounds_error=False, fill_value="extrapolate")(z_std)
G2_itp = interp1d(z, G2, kind="linear", bounds_error=False, fill_value="extrapolate")(z_std)

# Interpolate density
rho_profile_itp = interp1d(z, rho_profile, kind="linear", bounds_error=False, fill_value="extrapolate")(z_std)[:,None]

## Apply LRF to generate radiative heating

### Convert basis with EOFs

In [36]:
# convert G1 and G2 to omega field
G1_omega = -G1 * rho_profile * 9.81 # 1000 -> 100
G2_omega = -G2 * rho_profile * 9.81

G1_omega_itp = np.interp(z_std, z, G1_omega) # 1000 -> 100
G2_omega_itp = np.interp(z_std, z, G2_omega) 

# decompose vertical mode into PCs

G1_pcs = G1_omega_itp[None, :] @ eof.T @ np.linalg.inv(eof @ eof.T)
G2_pcs = G2_omega_itp[None, :] @ eof.T @ np.linalg.inv(eof @ eof.T)

In [37]:
print(G1_pcs.shape)

(1, 5)


### Compute radiation with LRF

In [38]:
# compute radiation for moisture
lw_moisture = (q_lw_lrf @ mean_moist_itp[:,None])
sw_moisture = (q_sw_lrf @ mean_moist_itp[:,None])

# compute radiation for temperature
lw_temp1 = t_lw_lrf @ (G1_itp[:,None]*0.0065)
sw_temp1 = t_sw_lrf @ (G1_itp[:,None]*0.0065)
lw_temp2 = t_lw_lrf @ (G2_itp[:,None]*0.0065) # 1000 -> 100
sw_temp2 = t_sw_lrf @ (G2_itp[:,None]*0.0065)

# compute cloud radiation with vertical motion

lw_cld1 = ( (w_lw_lrf @ G1_pcs.T).T @ eof).T
sw_cld1 = ( (w_sw_lrf @ G1_pcs.T).T @ eof).T # 1000 -> 100
lw_cld2 = ( (w_lw_lrf @ G2_pcs.T).T @ eof).T
sw_cld2 = ( (w_sw_lrf @ G2_pcs.T).T @ eof).T


## Decompose heating to obtain Coefficients

In [39]:
# stack different modes
modes = np.stack([G1_itp*0.0065, G2_itp*0.0065], axis=-1)



lw_moisture1, lw_moistur2 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * lw_moisture))
sw_moisture1, sw_moistur2 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * sw_moisture))

lw_temp11, lw_temp12 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * lw_temp1))
sw_temp11, sw_temp12 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * sw_temp1))
lw_temp21, lw_temp22 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * lw_temp2))
sw_temp21, sw_temp22 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * sw_temp2))

lw_cld11, lw_cld12 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * lw_cld1))
sw_cld11, sw_cld12 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * sw_cld1))
lw_cld21, lw_cld22 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * lw_cld2))
sw_cld21, sw_cld22 = np.linalg.inv(modes.T @ modes) @ (modes.T @ (rho_profile_itp * sw_cld2))

In [40]:
print("moisture-LW: ", lw_moisture1, lw_moistur2)
print("moisture-SW: ", sw_moisture1, sw_moistur2)
print("temp1-LW: ", lw_temp11, lw_temp12)
print("temp1-SW: ", sw_temp11, sw_temp12)
print("temp2-LW: ", lw_temp21, lw_temp22)
print("temp2-SW: ", sw_temp21, sw_temp22)
print("cloud1-LW: ", lw_cld11, lw_cld12)
print("cloud1-SW: ", sw_cld11, sw_cld12)
print("cloud2-LW: ", lw_cld21, lw_cld22)
print("cloud2-SW: ", sw_cld21, sw_cld22)


moisture-LW:  [5.5453094] [9.12231607]
moisture-SW:  [1.76369633] [-4.24707196]
temp1-LW:  [-0.02137107] [-0.00319688]
temp1-SW:  [5.89937197e-05] [-0.00016805]
temp2-LW:  [-0.0053464] [-0.03675032]
temp2-SW:  [7.83015985e-05] [7.13741032e-05]
cloud1-LW:  [3598.79000636] [7072.13322283]
cloud1-SW:  [2113.35769062] [-3526.41252402]
cloud2-LW:  [-1453.08302447] [-2495.18098186]
cloud2-SW:  [-108.8704286] [1550.53777944]


## Verification

In [41]:

def plot_all_rad_profiles_3x2(
    panels,
    levels,
    outpath,
    xlabel="Radiative Heating Rate (K/day)",
):
    """
    panels: list of dicts, each dict contains
        title, exact_lw, approx_lw, exact_sw, approx_sw
    levels: (nlev,)
    """

    def _vec(x):
        x = np.asarray(x)
        return x[:, 0] if (x.ndim == 2 and x.shape[1] == 1) else x

    # ---------- global x-limits ----------
    max_limit = 0.0
    for p in panels:
        for k in ("exact_lw", "approx_lw", "exact_sw", "approx_sw"):
            arr = _vec(p[k])
            max_limit = max(max_limit, np.nanmax(np.abs(arr[1:])))

    xlim = (-1.1 * max_limit, 1.1 * max_limit)

    # ---------- figure ----------
    fig, axes = plt.subplots(
        nrows=3, ncols=2,
        figsize=(10, 12),
        # sharex=True, sharey=True
    )

    axes = axes.flatten()

    for i, (ax, p) in enumerate(zip(axes, panels)):
        exact_lw  = _vec(p["exact_lw"])
        approx_lw = _vec(p["approx_lw"])
        exact_sw  = _vec(p["exact_sw"])
        approx_sw = _vec(p["approx_sw"])

        ax.plot(exact_lw,  levels, "k-",  label="Exact LW")
        ax.plot(approx_lw, levels, "k--", label="Approx LW")
        ax.plot(exact_sw,  levels, "-",  color="#1f77b4", label="Exact SW")
        ax.plot(approx_sw, levels, "--", color="#1f77b4", label="Approx SW")

        ax.set_title(p["title"], loc="left", fontsize=12)

        ax.spines["right"].set_visible(False)
        ax.spines["top"].set_visible(False)
        ax.minorticks_on()

        # ax.set_xlim(*xlim)
        ax.set_ylim(1000, 100)

    # ---------- turn off unused panel ----------
    for j in range(len(panels), len(axes)):
        axes[j].axis("off")

    # ---------- labels ----------
    axes[4].set_xlabel(xlabel)
    axes[0].set_ylabel("Pressure (hPa)")
    axes[2].set_ylabel("Pressure (hPa)")
    axes[4].set_ylabel("Pressure (hPa)")

    # ---------- single legend ----------
    handles, labels = axes[0].get_legend_handles_labels()
    fig.legend(
        handles, labels,
        loc="upper center",
        ncol=4,
        frameon=False
    )

    fig.tight_layout(rect=[0, 0, 1, 0.95])
    fig.savefig(outpath, dpi=300)
    plt.close(fig)

levels = np.linspace(1000, 100, 37)

panels = [
    dict(
        title="(a) Moisture",
        exact_lw=lw_moisture * rho_profile_itp,
        exact_sw=sw_moisture * rho_profile_itp,
        approx_lw=(lw_moisture1*G1_itp*0.0065 + lw_moistur2*G2_itp*0.0065)[:, None],
        approx_sw=(sw_moisture1*G1_itp*0.0065 + sw_moistur2*G2_itp*0.0065)[:, None],
    ),
    dict(
        title="(b) T₁",
        exact_lw=lw_temp1,
        exact_sw=sw_temp1,
        approx_lw=(lw_temp11*G1_itp*0.0065 + lw_temp12*G2_itp*0.0065)[:, None] / rho_profile_itp,
        approx_sw=(sw_temp11*G1_itp*0.0065 + sw_temp12*G2_itp*0.0065)[:, None] / rho_profile_itp,
    ),
    dict(
        title="(c) T₂",
        exact_lw=lw_temp2,
        exact_sw=sw_temp2,
        approx_lw=(lw_temp21*G1_itp*0.0065 + lw_temp22*G2_itp*0.0065)[:, None] / rho_profile_itp,
        approx_sw=(sw_temp21*G1_itp*0.0065 + sw_temp22*G2_itp*0.0065)[:, None] / rho_profile_itp,
    ),
    dict(
        title="(d) Cloud 1",
        exact_lw=lw_cld1,
        exact_sw=sw_cld1,
        approx_lw=(lw_cld11*G1_itp*0.0065 + lw_cld12*G2_itp*0.0065)[:, None] / rho_profile_itp,
        approx_sw=(sw_cld11*G1_itp*0.0065 + sw_cld12*G2_itp*0.0065)[:, None] / rho_profile_itp,
    ),
    dict(
        title="(e) Cloud 2",
        exact_lw=lw_cld2,
        exact_sw=sw_cld2,
        approx_lw=(lw_cld21*G1_itp*0.0065 + lw_cld22*G2_itp*0.0065)[:, None] / rho_profile_itp,
        approx_sw=(sw_cld21*G1_itp*0.0065 + sw_cld22*G2_itp*0.0065)[:, None] / rho_profile_itp,
    ),
]


plot_all_rad_profiles_3x2(
    panels,
    levels,
    outpath="/home/b11209013/2025_Research/MSI/Fig/Rad/all_components_3x2.png",
)